Credits:<br>
https://www.cnblogs.com/tolshao/p/buildingarecurrentneuralnetworkstepbystepv3b.html
https://datascience-enthusiast.com/DL/Building_a_Recurrent_Neural_Network-Step_by_Step_v1.html

# Introduction to Recurrent Neural Network

Everytime I come back to some NLP problem i realize that i need to cover RNN/LSTM from beginning again. So i decided to make this notebook which talks about RNN and atleast points to things that need to be understood to understand a working example of RNN/LSTM.<br><br>
In short this notebook covers the following 

- Why RNN and what kind of problem could be solved using RNN
- RNN architecture unrolled with time explanation
- RNN forward propagation eqn with example weights
- RNN coding example includes building a RNN cell from scratch and then use Pytorch to implement the same.
- RNN backpropagation and it's shortcomings(like back propagation through time(BPTT) and Truncated BPTT)

## Why RNN

In a problem like language translate, say you are translating from english to bengali, the input and output length could be different. So in this kind of problem we can't use a traditional CNN/ANN directly because the input size is variable.

Also if you want to predict the next word in a sentence it is important to know the words before it(context). So we need a network which has "memory" which captures information about what has been calculated so far.

## What is RNN
A recurrent neural network is a neural network that is specialized for processing a sequence of data x(t)= x(1), . . . , x(τ) with the time step index t ranging from 1 to τ.


# RNN architecture

Rnn architecture could be divided into 2 parts.RNN cell and the RNN model where multiple RNN cells are connected

## RNN Cell:

![RNN Cell](./images/rnn_cell.png)

**Figure 3**: Basic RNN cell. Takes as input  x⟨t⟩  (current input) and  a⟨t−1⟩  (previous hidden state containing information from the past), and outputs  a⟨t⟩  which is given to the next RNN cell and also used to predict  y⟨t⟩

Wax : Weight matrix multiplying the input the hidden layer.<br>
Waa : Weight matrix multiplying the previous the hidden layer(h(t-1)) to current hidden layer(ht)<br>
Wya : Weight matrix multiplying the output of the hidden layer to the output??(why can't we take the op directly)<br>
xt  : One Input sequence to the rnn cell at timestep t   <br>


## RNN Example 1:

Since you now have a basic idea, let’s break down the execution process with an example. Say your batch size is 6, RNN size is 7(so basically a(t) is 7 which means we are using a hidden layer of size 7),<br>
the number of time steps/segments you would include in one input 5(each time step could be thought as processing of one word which has a embedding dim or feature size 3) <br>
and the number of features in one time step is 3. <br>
If this is the case, your input tensor (matrix) shape for one batch would look something like this:
Tensor shape of one batch = (6,5,3)
The data inside a batch would look something like this:

![RNN model](./images/rnn_input.jpeg)

But when the RNN starts to process the data it will unroll and produce outputs as shown below:

<div>
<img src="./images/rnn_unroll.png" width="1000" height="1000" alt="rnn_unroll"/>
</div>

<!-- ![RNN model](./images/rnn_unroll.png){height=400px width=500px} -->

Couple of things to notice from the unrolled rnn image above:

The input size is (6,5,3) --> (batch_size,time_step,feature_length). let's say we are feeding something like "My name is Trinanjan", here we have 4 words, so time_step is 4 and each word could be represented by some embedding and we have to specify that dimension. and if we process a series of sentences then we have to specify batch size.<br>

The output size is (6,5,7) --> (batch_size,time_step,rnn_output)

we are feeding one input of size (1,3) at time step t. Now let's say the hidden layer size is 7.That is essentially we are using 7 nodes in the hidden layer. In this picture above we can see that the rnn output size is also same i.e 7 but it could vary. Let's say you define RNN hidden layer size as 100 and you have 10 output classes. In that case the Ow weight matix will have the dimension such that it comes out 10 from 100.

Also let's say we have one output at time t-1 , now when I pass this to the next layer hidden state notice i use hw[or Waa] weight to multiply and then the output goes to the next layer. I was wondering why can't i just pass it to the next layer directly. I guess we need to use these weight because weights are the one that is being learned.

So to sum up we have 3 sets of weight hw(Waa) then hx(Wax) and hy(Wya). Important thing to remember here is that these weights are shared meaning we don't use separate sets of weight at each time step.Other we will have huge number of parameters.


In [4]:
## NOTE : according to the picture above we have 3 weight matrices. 
# one for input to rnn hidden layer
# one for rnn hidden output to the next rnn timestep hidden output
# one for rnn output to convert to hidden state output to final oputput as you require
# ideally we should have 3 biases but in this example we are using only 2
# this example shows the equation with all 3 biases https://towardsdatascience.com/all-you-need-to-know-about-rnns-e514f0b00c7c

In [34]:
import numpy as np
import torch
np.random.seed(1)

# RNN One cell Forward Pass

In [58]:
# RNN one cell forward propagation

def rnn_cell_forward(xt,a_prev,parameters):
    """
    Implements a single forward step of RNN-cell as described in 
    Figure 3
    
    Arguments:
    xt -- input data at timestep "t", numpy array of shape(n_x,m)
    a_prev -- Hidden state at timestep "t-1", numpy array of shape(n_a,m)
    
    parameters -- python dict containing :
        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a,n_x)
        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y,n_a)
        ba  -- Bias for hidden layer, numpy array of shape (n_a,1)
        by  -- Bias relating the hidden-state to the output, numpy array of shape (n_y,1)
    
    Returns:
    a_next -- next hidden state, shape (n_a,m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    chche -- tuple of values needed for the backward pass, contains (a_next, a_prev, xt, parameters)
    
    """
    
    # Retrieve params from "parameters" dict
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    ### MAIN CODE ###
    
    # compute next activation state using the formula given above
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)
    # compute output of the current cell using the formula given above
    yt_pred = torch.nn.Softmax(np.dot(Wya, a_next) + by)
    yt_pred =  yt_pred.dim
    
    ### END OF MAIN ###
    
    # store values you need for backward propagation in cache
    cache = (a_next,a_prev,xt,parameters)
    
    return a_next, yt_pred, cache

In [60]:
m = 10 # number_of_example
input_feature_size = 3 # each word could be represented to a vector of size 3
hidden_size = 5 # hidden layer dimensions
output_size = 2 # how many output you want
xt = np.random.randn(input_feature_size,m) # 10 examples with each having a feature size of 3 
a_prev = np.random.randn(5,m) # m examples with 5 nodes
Waa = np.random.randn(hidden_size,hidden_size) # hidden to hidden
Wax = np.random.randn(hidden_size,input_feature_size) # input to hidden
Wya = np.random.randn(output_size,hidden_size) # hidden to output
ba = np.random.randn(hidden_size,1) # hidden to input
by = np.random.randn(output_size,1) # hidden to output
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt,a_prev,parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", a_next.shape)
print("yt_pred[1] =", yt_pred)
print("yt_pred.shape = ", yt_pred.shape)

a_next[4] =  [ 0.06941064  0.02212489  0.43926328  0.78063373  0.95994735  0.82543454
 -0.98515698 -0.98667449  0.54886588 -0.05653418]
a_next.shape =  (5, 10)
yt_pred[1] = [[ 0.9092852  -1.59848365  1.36423616 -0.57958794 -0.15054085 -0.66888275
  -2.65403086  0.16586335 -0.94538287  0.45756809]
 [ 1.78534977  1.4748327   1.36450624  3.15882046  3.19210891  2.53689657
   0.694187    0.11218914  2.34380738  1.0752556 ]]
yt_pred.shape =  (2, 10)


In [63]:
## Need to go through ANN weight initialization dimensions what does each row and column represent 
## each coulmn is one output for one example or each row is one output for one example 
## check out andrew ng lectures

# Rnn Full Forward Pass

You can see RNN as the repetion of of the cell you've just built. If your input sequence of data is carried over 10 time steps, then you will copy thr RNN cell 10 times. Each cell takes as input the hidden state from the previous cell (a(t-1)) and the current timestep xt. we have output as yt and hidden state output as at

## RNN model unrolled in time:

![RNN model](./images/rnn_model.png)
**Figure 3**: Basic RNN. The input sequence  x=(x⟨1⟩,x⟨2⟩,...,x⟨Tx⟩)  is carried over  Tx  time steps. The network outputs  y=(y⟨1⟩,y⟨2⟩,...,y⟨Tx⟩) .

Instructions:
    1. Create a vector of zeros(a) that will store all the hidden states computed by RNN.
    2. Initialize the "next" hidden state as a0 as zero matrix. this will as first timestep previous input 
    3. Start looping over each time step and each timestep should call the rnn_cell_forward

In [66]:
# RNN full forward propagation

def rnn_forward(xt,a0,parameters):
    
    """
    Implements a forward propagation as desribed in the figure above
    
    Arguments:
    x -- input data for timesteps T_x  numpy array of shape(n_x,m,T_x)
    a0 -- initial hidden state, numpy array of shape(n_a,m)
    
    Parameters -- python dict containing :
        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a,n_x)
        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y,n_a)
        ba  -- Bias for hidden layer, numpy array of shape (n_a,1)
        by  -- Bias relating the hidden-state to the output, numpy array of shape (n_y,1)
    
    Returns:
    a --Hidden states for every time-step, numpy array of shape (n_a,m,T_x)
    yt_pred -- prediction for every time-step, numpy array of shape (n_y, m,T_x)
    chche -- tuple of values needed for the backward pass, contains (list of caches, same as timesteps)
    
    """ 
    # initialize caches which will contail the list of all caches
    caches = []
    
    # retrieve dimensions from shapes of x and Wy
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    ### MAIN CODE HERE ###
    
    # initialize "a" and "y" with zeros 
    a = np.zeros((n_a,m,T_x)) 
    y_pred = np.zeros((n_y,m,T_x))
    a_next = a0
    
    # loop over time-steps
    for t in range(T_x):
        # update next hidden state, compute prediction, get the cache
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t],a_next,parameters)
        # save the value of the new "next" hidden state in a
        a[:,:,t] = a_next
        # save the value of pred
        y_pred[:,:,t] = yt_pred
        # save cache
        caches.append(cache)
    # store values for backpropagation
    caches = (caches,x)
    return a, y_pred, caches
    ### MAIN ENDS HERE ###
    

In [67]:
m = 10 # number_of_example
input_feature_size = 3 # each word could be represented to a vector of size 3
hidden_size = 5 # hidden layer dimensions
output_size = 2 # how many output you want\
time_step_size = 4 # time steps

x = np.random.randn(input_feature_size,m,time_step_size) # 10 examples with each having a feature size of 3 
a0 = np.random.randn(5,m) # m examples with 5 nodes
Waa = np.random.randn(hidden_size,hidden_size) # hidden to hidden
Wax = np.random.randn(hidden_size,input_feature_size) # input to hidden
Wya = np.random.randn(output_size,hidden_size) # hidden to output
ba = np.random.randn(hidden_size,1) # hidden to input
by = np.random.randn(output_size,1) # hidden to output
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a, y_pred, caches = rnn_forward(x,a0,parameters)
print("a[4][1] = ", a[4][1])
print("a.shape = ", a.shape)
print("y_pred[1][3] =", y_pred[1][3])
print("y_pred.shape = ", y_pred.shape)
print("caches[1][1][3] =", caches[1][1][3])
print("len(caches) = ", len(caches))

a[4][1] =  [ 7.84026820e-01 -9.81241829e-01 -2.99758431e-04 -9.89749262e-01]
a.shape =  (5, 10, 4)
y_pred[1][3] = [-0.75022292  3.11688946  2.97666446 -1.9297936 ]
y_pred.shape =  (2, 10, 4)
caches[1][1][3] = [-1.04936032  0.76025175 -0.7970639  -0.51758053]
len(caches) =  2
